In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
# Data Load
data_path = 'data/t_distribution_data.csv'
data = pd.read_csv(data_path)

In [3]:
# Data Preprocessing
le = LabelEncoder()
data['id'] = le.fit_transform(data['id'])
clusters = sorted(list(set(data['id'].values))) # unique cluster id

In [4]:
# Torch Dataset Class
class SIMDataset(Dataset):
    def __init__(self, data, targets, clusters):
        self.data = data
        self.targets = targets
        self.clusters = clusters
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.targets[index], self.clusters[index]

x_col = ['id', 'x'] # input columns
y_col = ['y'] # target columns

# Train-Test Split
SEED = 42
train, test = train_test_split(data, test_size=0.333, random_state=SEED, stratify=data['id'])

# Dataset Creation
train_dataset = SIMDataset(train[x_col].values, train[y_col].values, train['id'].values)
test_dataset = SIMDataset(test[x_col].values, test[y_col].values, test['id'].values)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Define PyTorch MeNet Model
class MeNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=[20, 5]):
        super(MeNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# Model Training
def train_menet(model, train_loader, n_clusters, device, epochs=100, lr=0.001, patience=10):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    # Initialize Random Effects
    output_dim = model.fc2.weight.data.size(0)  # Output dimension from fc2 weights
    b_hat = torch.zeros(n_clusters, output_dim, device=device)
    D_hat = torch.eye(output_dim, device=device)
    sig2e_est = 1.0

    best_loss = float('inf')
    wait = 0

    for epoch in tqdm(range(epochs)):
        model.train()
        epoch_loss = 0
        for batch in train_loader:
            X_batch, y_batch, cluster_ids = batch
            X_batch, y_batch = X_batch.to(device).float(), y_batch.to(device).float()

            # Forward pass to get Z (feature map)
            with torch.no_grad():
                feature_map = model.fc2(model.relu(model.fc1(X_batch)))  # Shape: [batch_size, hidden_dim[1]]
            
            # Debugging Z shape
            # print(f"Feature Map shape: {feature_map.shape}, Expected shape: [batch_size, {output_dim}]")

            # E-Step: Update Random Effects
            for cluster_id in range(n_clusters):
                indices = (cluster_ids == cluster_id).nonzero(as_tuple=True)[0]
                if indices.numel() == 0:
                    continue

                Z_i = feature_map[indices]  # Feature map for the current cluster
                y_i = y_batch[indices].squeeze(-1)  # Remove extra dimension, Shape: [len(indices)]
                f_hat_i = model(X_batch[indices]).squeeze()  # Shape: [len(indices)]

                # Debugging shapes
                # print(f"Z_i shape: {Z_i.shape}, D_hat shape: {D_hat.shape}, y_i shape: {y_i.shape}, f_hat_i shape: {f_hat_i.shape}")

                # Ensure dimensions match for matrix multiplication
                V_hat_i = Z_i @ D_hat @ Z_i.T + sig2e_est * torch.eye(Z_i.size(0), device=device)
                V_hat_inv_i = torch.linalg.inv(V_hat_i)

                # Expand (y_i - f_hat_i) to 2D for matrix multiplication
                residual = (y_i - f_hat_i).unsqueeze(-1)  # Shape: [len(indices), 1]

                # Update b_hat
                b_hat[cluster_id] = (D_hat @ Z_i.T @ V_hat_inv_i @ residual).squeeze(-1)

            # M-Step: Update Fixed Effects
            optimizer.zero_grad()
            y_pred = model(X_batch).squeeze()
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        epoch_loss /= len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs} : Loss {epoch_loss:.4f}")

        # Early Stopping
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping triggered")
                break

    return model, b_hat, sig2e_est

In [19]:
# Model Inference
def menet_predict(model, test_loader, b_hat, n_clusters, device):
    """
    Perform inference using the trained MeNet model with random effects.

    Args:
        model: Trained PyTorch model.
        test_loader: DataLoader for the test dataset.
        b_hat: Tensor containing random effect estimates for each cluster.
        n_clusters: Number of clusters (unique IDs).
        device: Device for computation (CPU or GPU).

    Returns:
        predictions: NumPy array of predictions for the test dataset.
    """
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in test_loader:
            X_batch, _, cluster_ids = batch
            X_batch = X_batch.to(device).float()

            # Forward pass through the model to compute fixed effects
            y_pred = model(X_batch).squeeze()  # Shape: [batch_size]

            # Apply random effects to each cluster
            for cluster_id in range(n_clusters):
                indices = (cluster_ids == cluster_id).nonzero(as_tuple=True)[0]
                if indices.numel() == 0:
                    continue

                # Apply random effects adjustment for the current cluster
                feature_map = model.fc2(model.relu(model.fc1(X_batch)))  # Shape: [batch_size, hidden_dim[1]]
                Z_i = feature_map[indices]  # Feature map for the current cluster
                adjustment = Z_i @ b_hat[cluster_id].unsqueeze(-1)  # Shape: [len(indices), 1]
                y_pred[indices] += adjustment.squeeze(-1)

            predictions.extend(y_pred.cpu().numpy())  # Convert predictions to NumPy

    return np.array(predictions)

In [8]:
# Main Execution
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = len(x_col)
n_clusters = len(clusters)

In [9]:
# Model Initialization
model = MeNet(input_dim).to(device)

In [10]:
# Train the Model
model, b_hat, sig2e_est = train_menet(model, train_loader, n_clusters, device, epochs=100)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([27, 1])) that is different to the input size (torch.Size([27])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  9%|▉         | 9/100 [00:00<00:02, 38.15it/s]

Epoch 1/100 : Loss 0.0499
Epoch 2/100 : Loss 0.0213
Epoch 3/100 : Loss 0.0144
Epoch 4/100 : Loss 0.0120
Epoch 5/100 : Loss 0.0110
Epoch 6/100 : Loss 0.0106
Epoch 7/100 : Loss 0.0104
Epoch 8/100 : Loss 0.0102
Epoch 9/100 : Loss 0.0100


 18%|█▊        | 18/100 [00:00<00:02, 37.43it/s]

Epoch 10/100 : Loss 0.0101
Epoch 11/100 : Loss 0.0101
Epoch 12/100 : Loss 0.0100
Epoch 13/100 : Loss 0.0100
Epoch 14/100 : Loss 0.0100
Epoch 15/100 : Loss 0.0100
Epoch 16/100 : Loss 0.0101
Epoch 17/100 : Loss 0.0100
Epoch 18/100 : Loss 0.0101


 23%|██▎       | 23/100 [00:00<00:01, 38.86it/s]

Epoch 19/100 : Loss 0.0100
Epoch 20/100 : Loss 0.0099
Epoch 21/100 : Loss 0.0100
Epoch 22/100 : Loss 0.0100
Epoch 23/100 : Loss 0.0099
Epoch 24/100 : Loss 0.0099
Epoch 25/100 : Loss 0.0100
Epoch 26/100 : Loss 0.0098
Epoch 27/100 : Loss 0.0099


 32%|███▏      | 32/100 [00:00<00:01, 34.24it/s]

Epoch 28/100 : Loss 0.0099
Epoch 29/100 : Loss 0.0099
Epoch 30/100 : Loss 0.0099
Epoch 31/100 : Loss 0.0099
Epoch 32/100 : Loss 0.0102
Epoch 33/100 : Loss 0.0098


 35%|███▌      | 35/100 [00:00<00:01, 35.49it/s]

Epoch 34/100 : Loss 0.0099
Epoch 35/100 : Loss 0.0099
Epoch 36/100 : Loss 0.0098
Early stopping triggered


In [20]:
# Predict
predictions = menet_predict(
    model=model,
    test_loader=test_loader,
    b_hat=b_hat,
    n_clusters=n_clusters,
    device=device,
)

# Print predictions
print("Predictions:", predictions)

Predictions: [ 0.026841    0.02152333  0.01048341  0.03317363  0.02105941  0.00579274
  0.02823902  0.01614921  0.00978274  0.03449761  0.03274583  0.03394176
  0.02439539  0.01868584  0.00238098  0.02549402  0.0297981   0.01108495
  0.02497786  0.06573636 -0.01682149  0.00842068  0.01425993  0.11389747
  0.01766851 -0.04875698  0.11411142  0.01639492  0.02290808  0.05536727
  0.01338203  0.0078006   0.01840146  0.01512287 -0.05439264  0.02384457
  0.02457746  0.11450274  0.04425842  0.01475005  0.0115121   0.07566318
  0.0304903   0.02709374  0.02448831  0.01368714  0.03750501  0.03213113
  0.09931414  0.01891589  0.04129791  0.03705351  0.04270222  0.02911329
  0.04258706  0.03589354  0.11823933  0.03963827  0.00895112  0.05186625
  0.0960172   0.03316765  0.03824171  0.02355003  0.02866612  0.03104725
 -0.00108824  0.01728559  0.01283209  0.02488139  0.04201642  0.02719539
  0.02186912  0.08920351  0.04724576  0.03582845  0.06578904  0.00704966
  0.00976964  0.00932837  0.00993091  

In [21]:
# Calculate evaluation metrics
def evaluate_predictions(y_true, y_pred):
    """
    Evaluate predictions using MSE, MAE, MAPE, and MRPE.
    
    Args:
        y_true (np.array): Ground truth values.
        y_pred (np.array): Predicted values.
    
    Returns:
        metrics (dict): Dictionary of evaluation metrics.
    """
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # Avoid dividing by zero
    mrpe = np.mean(np.abs((y_true - y_pred) / y_pred)) * 100

    return {
        "MSE": mse,
        "MAE": mae,
        "MAPE": mape,
        "MRPE": mrpe
    }

# Ground truth (y_test) should be prepared from your test dataset
y_test = test[y_col].values.flatten()

# Evaluate predictions
metrics = evaluate_predictions(y_test, predictions)

# Print metrics
print("===== Evaluation Metrics =====")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

===== Evaluation Metrics =====
MSE: 0.0125
MAE: 0.0689
MAPE: 63799.8312
MRPE: 266.0444
